In [ ]:
def format_reward_func(prompts, completions, **kwargs):
    """
    Enhanced reward function that checks format and penalizes numbers
    presented as strings in the macro state section
    """
    step = kwargs.get("step", 0)
    
    # Basic format pattern
    pattern = r"<macro state>.*?</macro state>.*?<reasoning>.*?</reasoning>.*?<positioning>.*?</positioning>"
    
    # Pattern to detect numbers written as strings in macro state
    number_as_string_pattern = r'<macro state>.*?(\b(?:one|two|three|four|five|six|seven|eight|nine|ten|hundred|thousand|million|billion|point|percent|[0-9]+\.[0-9]+%?)\b).*?</macro state>'
    
    completion_contents = [
        completion[0]["content"] if isinstance(completion[0], dict) else completion[0] 
        for completion in completions
    ]
    
    rewards = []
    format_valid_count = 0
    number_as_string_count = 0
    
    for content in completion_contents:
        # Check basic format
        format_valid = re.search(pattern, content, re.DOTALL) is not None
        
        if format_valid:
            format_valid_count += 1
            
            # Check for numbers as strings in macro state
            has_number_as_string = re.search(number_as_string_pattern, content, re.DOTALL) is not None
            
            if has_number_as_string:
                number_as_string_count += 1
                # Penalize but still give some reward for correct format
                rewards.append(1.0)  # Reduced reward (from 3.0 to 1.0)
            else:
                # Full reward for correct format without numbers as strings
                rewards.append(3.0)
        else:
            # Penalty for incorrect format
            rewards.append(-1.0)